# Taylor expansion

Inspired by https://www.r-craft.org/r-news/taylor-series-with-python-and-sympy-revised/

In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

With `sympy` (symbolical python) we can manipulate mathematical expressions analytically. 

In [2]:
import sympy as sy
# Import all available functions (bad practice but ok for our example)
from sympy.functions import *
factorial = np.math.factorial
from ipywidgets import HBox, IntSlider, FloatSlider, VBox, Text, Layout

In [3]:
def taylor(function, x0, n, x = sy.Symbol('x')):
    p = 0
    for i in range(n+1):
        p = p + (function.diff(x, i).subs(x, x0))/(factorial(i))*(x - x0)**i
    return p

## Simple example

In [4]:
# def function
x = sy.Symbol('x')
f = exp(x)

# get taylor expansion (ana)
func = taylor(f, 0, 4)
# convert it to a numpy function
taylor_lambda = sy.lambdify(x, func, "numpy")
xs = np.linspace(-5, 5, 100)

fig, ax = figure('example')
ax.plot(xs, taylor_lambda(xs), label='T_4')
ax.plot(xs, np.exp(xs), 'k--', label='func')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Widget

In [5]:
def taylor_plot(f, fig_num):
    order = IntSlider(value=0, min=0, max=30, description='order')
    x0 = FloatSlider(value=0, min=-5, max=5, description='x0')
    
    fig, ax = figure(fig_num)
    x = sy.Symbol('x')
    N = 300
    x1 = np.linspace(-5, 5, N)
    func_lambda = sy.lambdify(x, f, "numpy")
    ax.plot(x1, func_lambda(x1), label = '$f(x)$')
    line, = ax.plot(x1, [func_lambda(x0.value)]*N, label='$f_0(x)$')
    text = Text(value=f'{func_lambda(x0.value)}', description='Function:', layout=Layout(width='80%'))
    L = ax.legend()
        
    def update_order(change):
        redraw(change.new, x0.value)

    def update_x0(change):
        redraw(order.value, change.new)

    def redraw(new_order, new_x0):
        func = taylor(f, new_x0, new_order)
        taylor_lambda = sy.lambdify(x, func, "numpy")
        line.set_data(x1, taylor_lambda(x1))
        L.get_texts()[1].set_text(f'$f_{{{new_order}}}(x)$')
        text.value = str(func)
        fig.canvas.draw()
        fig.canvas.flush_events()    
        
    order.observe(update_order, names='value')
    x0.observe(update_x0, names='value')
    return VBox([HBox([order, x0]), text])

You can get a list of the available functions here [https://docs.sympy.org/latest/modules/functions/index.html](https://docs.sympy.org/latest/modules/functions/index.html).

In [7]:
x = sy.Symbol('x')
# f = cos(x) + 2 * sin(x)
# f = 1 / (1 + x)
# f = besseli(x, 1)
# f = ln(x)
f = exp(-x**2)
taylor_plot(f, 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …